<a href="https://colab.research.google.com/github/Ditsuhi/ConvLSTM_Madrid/blob/main/ConvLSTMadrid_NitrogenDioxidePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import zipfile
from glob import glob
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error




#unzip data giving the path of certain dataset
# path = '/content/Jan_Jun_2019.zip'
path = '/content/Jan_Jun_2020.zip'

with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

airMet = glob("/content/content/csvFiles/*.csv")

In [4]:
#sort dataset with chronological order
def sortingFiles(eachFile):
    return int(eachFile) if eachFile.isdigit() else eachFile
def natural_keys(eachFile):
    return [ sortingFiles(c) for c in re.split('(\d+)',eachFile)]

sorted_airMet = sorted(airMet, key = natural_keys)

In [5]:
# the following list includes all columns' names, there are 10  columns in total. Depends analysis we will select certain columns and concat them
# ['# FID', ' NO2', ' UV', ' windSpeed', ' windDir', ' Temp', ' Humidity', ' Pressure', ' SolarRad', ' Prec']
df = [pd.read_csv(f, usecols=[' NO2', ' Temp', ' Humidity']).values for f in sorted_airMet]
df_all = np.asarray(df)

In [6]:
df_all.shape

(4368, 340, 3)

In [7]:
 
# split dataset to X and y (dependent and independent)
def split_sequence(sequence, time_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + time_steps
		# check if we are beyond the sequence
		if end_ix+time_steps > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix+time_steps]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)
 

 # define input sequence
raw_seq = df_all
# choose a number of time steps (there are two case of time lags: 12-hour and 24-hour)
time_steps = 12

X, y_total = split_sequence(raw_seq, time_steps)
#to take only NO2 from whole features as a target variable
y =y_total[:, :, 0]

In [9]:
X.shape, y.shape

((4344, 12, 340, 3), (4344, 340))

In [10]:
#split data to train and test sets

X_train_notNorm, X_test_notNorm, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle = False)

In [14]:
y_test.shape

(1434, 340)

In [11]:
# to normalise train data using MinMaxScaler
number_selected_columns = 3

scaler = MinMaxScaler(feature_range=(0, 1), copy = False)
train_Normalised = X_train_notNorm.reshape(-1, 340*number_selected_columns)
test_Normalised = X_test_notNorm.reshape(-1, 340*number_selected_columns)

train_scaled = scaler.fit_transform(train_Normalised)
test_scaled = scaler.transform(test_Normalised)

X_train = train_scaled.reshape(X_train_notNorm.shape[0], X_train_notNorm.shape[1], X_train_notNorm.shape[2], X_train_notNorm.shape[3])
X_test = test_scaled.reshape(X_test_notNorm.shape[0], X_test_notNorm.shape[1], X_test_notNorm.shape[2], X_test_notNorm.shape[3])

In [12]:
X_train.shape

(2910, 12, 340, 3)

In [15]:
#reshape data for further analysis

X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 340, number_selected_columns, 1))
y_train_reshaped = y_train.reshape((y_train.shape[0], 340, 1, 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 340, number_selected_columns, 1))
y_test_reshaped = y_test.reshape(y_test.shape[0], 340)

In [16]:
#create model architecture

model = keras.Sequential([keras.Input(shape=(None,  340, number_selected_columns, 1)),          
        layers.ConvLSTM2D(filters=16, kernel_size=(3, 3), padding="same", return_sequences=True, kernel_regularizer=l2(0.01)),   
        layers.Dropout(0.5),
        layers.BatchNormalization(),    
        layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=True),        
        layers.BatchNormalization(),
        layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=True),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(filters=16, kernel_size=(3, 3), padding="same", return_sequences=True),
        layers.BatchNormalization(),          
        layers.ConvLSTM2D(filters=1, kernel_size=(1, number_selected_columns), activation='relu')])


model.compile(optimizer="adam", loss='mse')
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, None, 340, 3, 16)  9856      
_________________________________________________________________
dropout (Dropout)            (None, None, 340, 3, 16)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, None, 340, 3, 16)  64        
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 340, 3, 32)  55424     
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 340, 3, 32)  128       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, None, 340, 3, 32)  73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 340, 3, 32)  1

In [ ]:
# Validation_split, it takes the last part of the training set
# run the model using earlystopping in order to escape overfitting


es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)
history = model.fit(X_train_reshaped, y_train_reshaped,  epochs=100, verbose=2, validation_split=0.1, shuffle=False, callbacks=[es])
# demonstrate prediction

In [ ]:
# calculate error in test set
yhat = model.predict(X_test_reshaped , verbose=1)
yhat_reshaped = yhat.reshape(y_test.shape[0], 340)


testScore = mean_squared_error(yhat_reshaped, y_test_reshaped, squared=False)
print('Test Score: %.2f RMSE' % (testScore))
